# Text analysis of restaurant reviews
## Find hamburgers!

In this notebook we analyze the Iens restaurant reviews for a city and a specific date.

In case gbq is not working you might need to update the python api client library:

```bash
sudo pip install --upgrade google-api-python-client
```

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq as gbq 
import json
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# hide warnings. `gbq.read_gbq()` gives some
import warnings
warnings.filterwarnings('ignore')

In [78]:
from pprint import pprint

In [2]:
# project specifics
PRIVATE_KEY = '../google-credentials/gsdk-credentials.json'
PROJECT_ID = json.load(open(PRIVATE_KEY))['project_id']

In [3]:
# dataset specifics
city = 'amsterdam'
date = '20180123'
bq_table = '_'.join(['iens.iens_comments', city, date])

In [4]:
!bq ls

  datasetId  
 ----------- 
  iens       
  tinkering  


In [5]:
!bq ls iens


              tableId                Type    Labels   Time Partitioning  
 ---------------------------------- ------- -------- ------------------- 
  iens_amsterdam_20171107            TABLE                               
  iens_amsterdam_20171201            TABLE                               
  iens_amsterdam_20171228            TABLE                               
  iens_amsterdam_20180123            TABLE                               
  iens_comments_amsterdam_20171228   TABLE                               
  iens_comments_amsterdam_20180123   TABLE                               
  iens_comments_diemen_20171227      TABLE                               
  iens_diemen_20171201               TABLE                               
  iens_diemen_20171227               TABLE                               


In [6]:
# dataset specifics (restaurants info)
bq_table_restaurants = '_'.join(['iens.iens', city, date])

## Reading from BigQuery

To load a BigQuery table into a Pandas dataframe, all you need is a query, the project_id, and a way to authenticate.

In [7]:
query = "SELECT * FROM {}".format(bq_table)

df = gbq.read_gbq(query, project_id=PROJECT_ID, private_key=PRIVATE_KEY)

Requesting query... ok.
Job ID: job_oKzLBusOwV2X60TQCvCWJNVKwg5p
Query running...
Query done.
Processed: 46.4 MB
Standard price: $0.00 USD

Retrieving results...
  Got page: 1; 18% done. Elapsed 10.91 s.
  Got page: 2; 42% done. Elapsed 17.49 s.
  Got page: 3; 68% done. Elapsed 24.47 s.
  Got page: 4; 86% done. Elapsed 31.05 s.
  Got page: 5; 100% done. Elapsed 35.4 s.
Got 94854 rows.

Total time taken 36.86 s.
Finished at 2018-01-24 09:16:31.


In [8]:
df.shape

(94854, 10)

# Analysis

In [9]:
df['date'] = pd.to_datetime(df['date'])
df['yearmonth'] = df['date'].dt.strftime('%Y-%m')
df['year'] = df['date'].dt.strftime('%y')
df['month'] = df['date'].dt.strftime('%m')

In [10]:
df.head().T

,0,1,2,3,4
name,Brasserie Van Dam,Kapitein Zeppos,Kapitein Zeppos,Kapitein Zeppos,Kapitein Zeppos
reserved_online,False,False,False,False,False
id,219761,220017,220017,220017,220017
rating_service,7,8,8,8,7
date,2015-07-08 00:00:00,2015-09-10 00:00:00,2012-09-25 00:00:00,2014-02-18 00:00:00,2013-07-06 00:00:00
rating_food,9,8,8,8,8
rating_decor,8,9,9,9,10
reviewer,Ben K.,Chili C.,Peter S.,Mart S.,peter1979 .
rating,8.3,8.3,8.3,8.3,8.3
comment,"Vooraf Vitello tonnato gegeten, was een verras...",Voor ons bezoek aan de Kleine Komedie nog even...,"Tja de ervaringen hieronder wisselen nogal, en...",alsje het niet weet loop je er zo aan voorbij...,Leuk restaurantje in een steegje met een heel ...


# Elastic search

To run Elasticsearch as a service:
+ `brew tap homebrew/services`
+ `brew services start elasticsearch`

You can check that it's up-and-running by examining the logs:
+ `tail -n 15 /usr/local/var/log/elasticsearch.log`


In [13]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [110]:
es.ping()

True

In [111]:
res = es.cat.health(v=True, h=['timestamp', 'cluster', 'status', 'node.total'])
print(res)

timestamp cluster            status node.total
12:30:38  elasticsearch_roel yellow          1



In [143]:
print(es.cat.indices(v=True))

health status index                uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   the_times_of_sklearn tKyiTez1RpK9mAb9YcHpjg   5   1      11314            0     18.4mb         18.4mb
yellow open   real_estate_listings OSxmQmoOQtu9dh4AcFBFNg   5   1          0            0      1.2kb          1.2kb



In [142]:
es.indices.delete(index='iens_comments')

{'acknowledged': True}

In [144]:
hyphens_and_apostrophes_strip = {
    "hyphens_and_apostrophes_strip": {
        "type": "mapping",
        "mappings": [ 
            "- => ' '",
            "' => "
        ]
    }
}

In [145]:
dutch_stop = {
    "dutch_stop": {
        "type": "stop",
        "stopwords": "_dutch_"
    }
}

In [146]:
analyzer = {
    "iens_comments_analyzer": {
        "type":           "custom",
        "char_filter":  [ "hyphens_and_apostrophes_strip" ],
        "tokenizer":      "standard",
        "filter":       [ "lowercase", "dutch_stop", "asciifolding" ]
    }
}

In [117]:
es.indices.create(index='iens_comments', ignore=400)

es.indices.close(index='iens_comments')

es.indices.put_settings(
    index='iens_comments', 
    body={
        "analysis": {
            "char_filter": hyphens_and_apostrophes_strip,
            "filter": dutch_stop,
            "analyzer": analyzer
        }
    }
)

es.indices.open(index='iens_comments')

{'acknowledged': True, 'shards_acknowledged': True}

In [119]:
es.indices.put_mapping(
    index='iens_comments',
    update_all_types=True,
    doc_type='article',
    body={
        "properties": {
            "comment": {
                "type": "text",
                "analyzer": "iens_comments_analyzer"
            },
            "name": {
                "type": "text"            
            },
        }
    }
)

{'acknowledged': True}

In [132]:
pprint(
    es.indices.get_mapping(index='iens_comments')['iens_comments']['mappings']
)

{'article': {'properties': {'comment': {'analyzer': 'iens_comments_analyzer',
                                        'type': 'text'},
                            'name': {'type': 'text'}}}}


In [133]:
print(es.cat.indices(v=True))

health status index                uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   the_times_of_sklearn tKyiTez1RpK9mAb9YcHpjg   5   1      11314            0     18.4mb         18.4mb
yellow open   real_estate_listings OSxmQmoOQtu9dh4AcFBFNg   5   1          0            0      1.2kb          1.2kb
yellow open   iens_comments        fadEUh-rTJulFe_cpFGXpg   5   1      94854            0     43.3mb         43.3mb



In [147]:
src

{'comment': 'Vooraf Vitello tonnato gegeten, was een verrassing, want eigenlijk niet alleen mooi op smaak, maar bijv ook gefrituurde kappertjes erop. Daarna hamburger met gegrilde groenten gehad. Mooi vlees, goed gemaakt, anders dan andere burgers, kortom ook prima. Wel grote portie, maar dat is geen nadeel. Bediening was prima: aardig maar gereserveerd. Geen arrogantie gezien, wel een trotse werknemer die het fijn vond dat het elke avond wel vol zit.',
 'name': 'Brasserie Van Dam'}

In [122]:
from elasticsearch.helpers import parallel_bulk
def generate_actions(n_comments):    
    for i in range(n_comments):
        src = {
            'comment': df.loc[i, 'comment'],
            'name': df.loc[i, 'name'],
#             'rating': df.loc[i, 'rating_food']    
        }

        yield {
            '_op_type': 'index',
            '_source': src 
        }

In [123]:
for success, info in parallel_bulk(
    client=es, 
    actions=generate_actions(len(df)),
    index='iens_comments', 
    doc_type='article',
    thread_count=4):
    if not success: print('Document insertion failed', info)

In [126]:
print(es.cat.indices(v=True))

health status index                uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   the_times_of_sklearn tKyiTez1RpK9mAb9YcHpjg   5   1      11314            0     18.4mb         18.4mb
yellow open   real_estate_listings OSxmQmoOQtu9dh4AcFBFNg   5   1          0            0      1.2kb          1.2kb
yellow open   iens_comments        fadEUh-rTJulFe_cpFGXpg   5   1      94854            0     43.3mb         43.3mb



In [125]:
res = es.search(index='iens_comments')
pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'KivyJ2EB_DuYwroOnjFz',
                    '_index': 'iens_comments',
                    '_score': 1.0,
                    '_source': {'comment': 'Bij Zeppos vooraf de vongole, '
                                           "mosselen en gamba's op Portugese "
                                           'wijze gegeten en daarna de vis van '
                                           'de dag. Het voorgerecht was om '
                                           'mijn vingers bij af te likken. '
                                           'Mijn tafelgenote had een kip met '
                                           'rozemarijn en aardappeltjes en die '
                                           'was ook erg lekker. De huiswijn is '
                                           'meer dan goed. Toe hadden we een '
                                           'chocoladesoufflé en een crème '
           

In [127]:
test_phrase = 'Wij hebben heerlijk gegeten bij Yamazato en een hele leuke avond gehad.'

In [128]:
res = es.indices.analyze(
    index='iens_comments',
    body={'analyzer': 'iens_comments_analyzer', 'text': test_phrase}
)

for token in res['tokens']:
    print(token['token'])

wij
heerlijk
gegeten
yamazato
hele
leuke
avond
gehad


In [139]:
res = es.search(
    index='iens_comments', 
    doc_type='article',
    body={
        "query": {
            "match": {
                "comment": "hamburger"
            }
        }
    }
)

In [140]:
for document in res['hits']['hits']:
    print('Score: ', document['_score'])
    pprint(document['_source'], indent=2)

Score:  7.956767
{ 'comment': 'Prima gegeten, hebben gekozen voor het broodje hamburger, was '
             'lekker! Er zat patat bij (staat niet op de menukaart bij '
             'hamburger), hamburger was een beste hap, dessert was ook geen '
             'ruimte meer voor    Hele vriendelijke bediening!',
  'name': 'Holland Casino Amsterdam'}
Score:  7.6797304
{ 'comment': 'Normaal gesproken zou ik een slechte eet ervaring niet zo graag '
             'via het internet verspreiden. Echter soms zijn er van die '
             'momenten, dat je iets uit frustratie wel doet. Ik heb vanavond '
             'een hamburger gehaald bij Burgerlijk. Na een kwartier buiten te '
             'hebben gewacht ben ik naar binnen gegaan om te vragen of mijn '
             'hamburger al bijna klaar zou zijn. Helaas waren ze nog niet '
             'begonnen.    Snel werd mijn hamburger in elkaar geduwd, en 10 '
             'minuten later stond ik buiten, om mijn hamburger later thuis op '
        

In [141]:
res = es.search(
    index='iens_comments', 
    doc_type='article',
    body={
        "query": {
            "bool": {
                "filter": {
                    "term": {
                        "comment": "hamburger"
                    }
                }
            }
        }
    }
)
res

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'kizyJ2EB_DuYwroOsCti',
    '_index': 'iens_comments',
    '_score': 0.0,
    '_source': {'comment': "Gezellig geluncht met collega, voordat we naar de Horecava gingen. Heerlijke sappige hamburger op een lekker vers broodje. We hadden de 'classic' genomen en de samenstelling/ingredienten van de hamburger was perfect. Ook de frietjes met de verse kruiden waren verrassend lekker. Goede prijs/kwaliteit verhouding op een leuke locatie. Dame van de bediening was bijzonder vriendelijk.",
     'name': 'Thrill Grill (Amsterdam Zuid)'},
    '_type': 'article'},
   {'_id': 'xCzyJ2EB_DuYwroOsDCf',
    '_index': 'iens_comments',
    '_score': 0.0,
    '_source': {'comment': 'Normaal neem ik de biefstuk met frietjes en salade. Wat erg lekker en mals is. Nu had ik een hamburger en dat was heel erg lekker. Voor 10 euro een hamburger op vers brood, een kleine salade erbij. Het was erg smaakvol.  Salade die

## TODO: kijk hoeveel % van reviews iets over hamburgers zegt